# Patient Readmission Prediction
_**Predict a patients readmission result using SageMaker's Linear-Learner binary classifier**_



## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data Ingestion and Feature Engineering](#Data-Ingestion-and-Feature-Engineering)
1. [Model Training](#Model-Training)
1. [Model Hosting](#Model-Hosting)
1. [Generating Predictions](#Generating-Predictions)
1. [Conclusions](#Conclusions)

---


## Background
This notebook illustrates how one can use SageMaker's algorithms for solving the problem of predicting patient readmissions. This notebook is reffered to in the (blog post paste link to the SPO blog). The purpose here is to use the data set that was preprocessed and is now stored in S3 (see blog for details) to build a predictve model of whether a patient will be readmitted or not. The data set will be used to illustrate

* Basic setup for using SageMaker.
* converting datasets to protobuf format used by the Amazon SageMaker algorithms and uploading to S3. 
* Training SageMaker's linear learner on the data set.
* Hosting the trained model.
* Scoring using the trained model.



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give learning and hosting access to your data. The snippet below will use the same role used by your SageMaker notebook instance, if you're using other.  Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.



In [83]:
import os
import boto3
from sagemaker import get_execution_role



role = get_execution_role()

bucket = 'diabetesdata'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'spo/sagemakermodel' 

Now we will import the python libraries we need.

In [78]:
import pandas as pd
import numpy as np
import io
import sagemaker.amazon.common as smac
import sagemaker 


---
## Data Ingestion and Feature Engineering

Data Source: The data source is the non-phi csv file that was created as part of the ETL job that was run in Glue. You can download it from the follwoing location: s3://diabetesdata/spo/nonphi/part-00000-ee59ca83-10ab-482b-990d-5d2a8adc4a9a-c000.csv

Let's read the data look at its contents.

In [79]:
df = pd.read_csv('Path to S3 file')

print (df.shape)
print (df.dtypes)
df.head(5)

(101766, 15)
admission_type           object
discharge_disposition    object
admission_source         object
time_in_hospital          int64
payer_code               object
medical_specialty        object
num_procedures            int64
num_medications           int64
number_outpatient         int64
number_emergency          int64
number_inpatient          int64
number_diagnoses          int64
insulin                  object
diabetesmed              object
readmission_result       object
dtype: object


,admission_type,discharge_disposition,admission_source,time_in_hospital,payer_code,medical_specialty,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,insulin,diabetesmed,readmission_result
0,Urgent,Discharged to home,Transfer from a hospital,4,MC,Cardiology,1,17,0,0,0,9,Down,Yes,Yes
1,Emergency,Discharged to home,Emergency Room,3,DM,InternalMedicine,1,11,1,2,1,9,No,No,Yes
2,Elective,Discharged to home,Physician Referral,12,MC,Psychiatry,0,18,0,0,0,9,No,Yes,Yes
3,Urgent,Discharged/transferred to SNF,Emergency Room,13,MC,Pulmonology,6,18,0,0,0,7,No,No,Yes
4,Elective,Discharged to home,Physician Referral,6,MC,Family/GeneralPractice,0,19,0,0,0,9,Down,Yes,Yes


#### Key observations:
* Data has 101766 observations and 15 columns.
* 8 fields are categorical and 7 fileds are integers.
* The last field , 'readmission_result', is an indicator of the actual outcome ('Yes' = Readmitted; 'No' = Not Readmitted). This will be our target attribute.


---
#### Convert categorical attributes to integers

We use the pandas get_dummies funtion to convert the catrgorical attributes into integers.


In [80]:
data_bin= pd.get_dummies(df, columns = [
    'admission_type',
    'discharge_disposition',
    'admission_source',
    'payer_code', 
    'medical_specialty', 
    'insulin', 
    'diabetesmed'])

print (data_bin.shape)

print (data_bin["readmission_result"].value_counts())

data_bin['readmission_result'] = np.where(data_bin['readmission_result'].str.contains('Yes'), 1, 0)

print (data_bin["readmission_result"].value_counts())


(101766, 156)
No     90399
Yes    11357
Name: readmission_result, dtype: int64
0    90399
1    11367
Name: readmission_result, dtype: int64


We can see that the resuting dataset has 90,399 records with no readmissions and 11357 records which resulted in readmissions

## Create Features and Labels
#### Split the data into 80% training and 20% validation.

In [81]:
rand_split = np.random.rand(len(data_bin))
train = rand_split < 0.8
test = rand_split >= 0.8

data_train = data_bin[train]
data_test = data_bin[test]


print (data_train.shape)
print (data_test.shape)



(81516, 156)
(20250, 156)


We now upload the formatted data into S3 as a CSV files for training and test data. 

In [84]:
data_train.to_csv("formatted_train.csv", sep=',') # save training data 
data_test.to_csv("formatted_test.csv", sep=',') # save test data

train_file = 'formatted_train.csv'
test_file= 'formatted_test.csv'

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/', train_file)).upload_file(train_file)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/', test_file)).upload_file(test_file)


We will now create:
    * The train_X matrix that contains the training data.
    * The train_y matrix that contains the training label, in our case readmission_result.

In [85]:
target_index_train= data_train.columns.get_loc("readmission_result")

train_y = data_train.iloc[:, target_index_train].as_matrix()
train_X = data_train.loc[:, data_train.columns != 'readmission_result'].as_matrix()


print (train_X.shape)
print (train_y.shape)

(81516, 155)
(81516,)


We see that our training dataset contains 81,416 records and 155 features. We will now use similar technique to split test data and label into matrix test_X and test_y as shown in the code below.

In [86]:
target_index_test= data_test.columns.get_loc("readmission_result")

test_y = data_test.iloc[:,target_index_test].as_matrix()
test_X = data_test.loc[:, data_test.columns != 'readmission_result'].as_matrix()


print (test_X.shape)
print (test_y.shape)

(20250, 155)
(20250,)


Now, we'll convert the datasets to the recordIO-wrapped protobuf format used by the Amazon SageMaker algorithms, and then upload this data to S3.  We'll start with training data.

In [92]:
train_file = 'linear_train.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype('float32'), train_y.astype('float32'))
f.seek(0)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'linear/train', train_file)).upload_fileobj(f)


Next we'll convert and upload the validation dataset.

In [88]:
test_file = 'linear_test.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, test_X.astype('float32'), test_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'linear/test', test_file)).upload_fileobj(f)


---
## Model Training

Now we can begin to specify our linear model.  Amazon SageMaker's Linear Learner is a pre packaged container that can be accessed to create an estimator. We provide hyperparameters to the estimator like:

- `feature_dim` Number of features in input data. In our case, we have 155 total features in our data.
- `predictor_type` Whether the target variable is binary classification or regression. In our case, we use binary classification as we are predicting the state of 1 (for a positive readmission result) or 0 (for a negative readmission result).
- `mini_batch_size` Mini batch size for data iterator, consisting of number of observations per mini batch. We choose 100 as a batch size for our training job.


### Specify container images used for training and hosting SageMaker's linear-learner

In [89]:
linear_containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}

Now let's kick off our training job in SageMaker's distributed, managed training. The sagemaker estimator is used to fit the training data stored on S3. The resultant model artifacts are stored in S3 at the end of the training job in the path specified in the output_location parameter.

In [90]:


output_location = 's3://{}/{}/linear/output'.format(bucket, prefix)

s3_train_data = 's3://{}/{}/linear/train/{}'.format(bucket, prefix, train_file)

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(linear_containers[boto3.Session().region_name],
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=155,
                           predictor_type='binary_classifier',
                           mini_batch_size=100)

linear.fit({'train': s3_train_data})


INFO:sagemaker:Creating training-job with name: linear-learner-2018-07-10-19-19-21-232


.....................
Docker entrypoint called with argument(s): train
[07/10/2018 19:22:45 INFO 139724623587136] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', u'num_calibration_samples': u'10000000', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', u'target_recall': u'0.8', u'num_models': u'32', u'early_stopping_patience': u'3', u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'_tuning_objective_metric': u'', u'early_stopping_tolerance': u'0.001', u'learning_rate': u'auto', u'_kvstore': u'auto', u'normalize_data': u'true', u'binary_classifier_model_selection_crit

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.334342119486054, "sum": 0.334342119486054, "min": 0.334342119486054}}, "EndTime": 1531250611.173695, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1531250611.173635}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3389413421958502, "sum": 0.3389413421958502, "min": 0.3389413421958502}}, "EndTime": 1531250611.173773, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1531250611.17376}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3344140163608855, "sum": 0.3344140163608855, "min": 0.3344140163608855}}, "EndTime": 1531250611.173824, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", 

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3322504684237615, "sum": 0.3322504684237615, "min": 0.3322504684237615}}, "EndTime": 1531250633.803543, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1531250633.803485}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.33829180417324134, "sum": 0.33829180417324134, "min": 0.33829180417324134}}, "EndTime": 1531250633.803621, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1531250633.803605}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3322896387884222, "sum": 0.3322896387884222, "min": 0.3322896387884222}}, "EndTime": 1531250633.803677, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Lea

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.33128580446769856, "sum": 0.33128580446769856, "min": 0.33128580446769856}}, "EndTime": 1531250655.094197, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1531250655.094138}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3377188008548292, "sum": 0.3377188008548292, "min": 0.3377188008548292}}, "EndTime": 1531250655.094272, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1531250655.094259}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3313548155123471, "sum": 0.3313548155123471, "min": 0.3313548155123471}}, "EndTime": 1531250655.094323, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Lea

...

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.330610134885355, "sum": 0.330610134885355, "min": 0.330610134885355}}, "EndTime": 1531250699.421287, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1531250699.421226}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3366477955484683, "sum": 0.3366477955484683, "min": 0.3366477955484683}}, "EndTime": 1531250699.421369, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 5}, "StartTime": 1531250699.421352}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3307966084275509, "sum": 0.3307966084275509, "min": 0.3307966084275509}}, "EndTime": 1531250699.421423, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner",

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3305152892949391, "sum": 0.3305152892949391, "min": 0.3305152892949391}}, "EndTime": 1531250721.646924, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1531250721.646862}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.336175284192606, "sum": 0.336175284192606, "min": 0.336175284192606}}, "EndTime": 1531250721.647009, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1531250721.646991}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.33078830039135515, "sum": 0.33078830039135515, "min": 0.33078830039135515}}, "EndTime": 1531250721.647062, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learne

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.33047368555127477, "sum": 0.33047368555127477, "min": 0.33047368555127477}}, "EndTime": 1531250744.511948, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1531250744.51189}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3357422882852379, "sum": 0.3357422882852379, "min": 0.3357422882852379}}, "EndTime": 1531250744.512023, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1531250744.512011}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3307808267909325, "sum": 0.3307808267909325, "min": 0.3307808267909325}}, "EndTime": 1531250744.512077, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Lear

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.330466246294829, "sum": 0.330466246294829, "min": 0.330466246294829}}, "EndTime": 1531250766.052601, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1531250766.052542}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.33534763154807995, "sum": 0.33534763154807995, "min": 0.33534763154807995}}, "EndTime": 1531250766.052674, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1531250766.052661}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.3307742088177453, "sum": 0.3307742088177453, "min": 0.3307742088177453}}, "EndTime": 1531250766.05271, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner

===== Job Complete =====
Billable seconds: 312


## Model Hosting
Now that we've trained our model, we can deploy it behind an Amazon SageMaker real-time hosted endpoint.  This will allow us to make predictions (or inference) from the model dyanamically. We first create an endpoint and deploy it behind a REST API by calling the deploy function.



In [93]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: linear-learner-2018-07-10-19-32-29-106
INFO:sagemaker:Creating endpoint with name linear-learner-2018-07-10-19-19-21-232


----------------------------------------------------------------------------!

## Generating Predictions
### Predict on Test Data

Now that we have our hosted endpoint, we can generate statistical predictions from it.  Let's predict on our test dataset to understand how accurate our model is.

There are many metrics to measure classification accuracy.  Common examples include include:
- Precision
- Recall
- F1 measure
- Area under the ROC curve - AUC
- Total Classification Accuracy 
- Mean Absolute Error

For our example, we'll keep things simple and use total classification accuracy as our metric of choice. We will also evalute  Mean Absolute  Error (MAE) as the linear-learner has been optimized using this metric, not necessarily because it is a relevant metric from an application point of view. We'll compare the performance of the linear-learner against a naive benchmark prediction which uses majority class observed in the training data set for prediction on the test data.


In [94]:
from sagemaker.predictor import csv_serializer, json_deserializer
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer


We'll now invoke the endpoint created by us to get predictions. We invoke the endpoint in a loop for each test data.

In [95]:
predictions = []
test_pred = []
for array in test_X:
    result = linear_predictor.predict(array)
    predictions += [r['predicted_label'] for r in result['predictions']]
    test_pred += [r['score'] for r in result['predictions']]
    
    

predictions = np.array(predictions)
test_pred = np.array(test_pred)

In [96]:
print (predictions.shape)
print (test_pred.shape)

(20250,)
(20250,)


We now have a prediction generated for each test record. 
Let's compare linear learner based mean absolute prediction errors from a baseline prediction which uses majority class to predict every instance.

In [97]:
test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(np.abs(test_y - np.median(train_y))) ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear,3))


Test MAE Baseline : 0.111
Test MAE Linear: 0.188


Let's compare predictive accuracy using a classification threshold of 0.5 for the predicted and compare against the majority class prediction from training data set

In [98]:
test_pred_class = (test_pred > 0.5)+0;
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class))*100
baseline_accuracy = np.mean((test_y == test_pred_baseline))*100

print("Prediction Accuracy:", round(prediction_accuracy,1), "%")
print("Baseline Accuracy:", round(baseline_accuracy,1), "%")

Prediction Accuracy: 88.9 %
Baseline Accuracy: 88.9 %


Finally, lets print the classification report and confusion matrix for our model

In [99]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print ("            -----Classification Report-----")
print (classification_report(test_y, test_pred_class))
print ("-Confusion Matrix-")
pd.DataFrame(confusion_matrix(test_y, test_pred_class, labels=[0, 1]))

            -----Classification Report-----
             precision    recall  f1-score   support

          0       0.89      1.00      0.94     18010
          1       0.50      0.02      0.03      2240

avg / total       0.85      0.89      0.84     20250

-Confusion Matrix-


,0,1
0,17974,36
1,2204,36


###### Run the cell below to delete endpoint once you are done.

In [100]:
sagemaker.Session().delete_endpoint(linear_predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: linear-learner-2018-07-10-19-19-21-232


---
## Conclusions

- Our linear model does a good job of predicting patient readmission and has an overall accuracy of close to 90%. We can re-run the model with different values of the hyper-parameters and see if we get improved prediction. Re-running the model with further tweaks to hyperparameters may provide more accurate out-of-sample predictions.
- We also did not do much feature engineering. We can create additional features by considering cross-product/intreaction of multiple features, squaring or raising higher powers of the features to induce non-linear effects, etc. If we expand the features using non-linear terms and interactions, we can then tweak the regulaization parameter to optimize the expanded model and hence generate improved forecasts.
- As a further extension, we can use many of non-linear models available through SageMaker such as XGBoost, MXNet etc.
